<img src="header_anwender.png" align="left"/>

# Anwendungsbeispiel Classification IRIS Flowers


Das Ziel des Beispieles ist es, die Gattung einer Iris Blume anhand von 4 Messgrößen zu schätzen. Dazu verwenden wir 
verschiedene Klassifikationsmethoden.
Wir verwenden einen Datensatz von Edgar Anderson oder R. Fischer aus dem Jahr 1936 [1][2]. Der Datensatz enthält 150 Samples mit jeweils
4 Messwerten petal length, petal width, sepal length, sepal width als Features sowie der korrekten Klasse als Label. 

```
[1] Edgar Anderson (1936). "The species problem in Iris". Annals of the Missouri Botanical Garden. 23 (3): 457–509. doi:10.2307/2394164. JSTOR 2394164.
[2] R. A. Fisher, “The use of multiple measurements in taxonomic problems,” Ann. Eugen., vol. 7, no. 2, pp. 179–188, 1936.
```

In [ ]:
#
# Abdrehen von Fehlermeldungen
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)



#
# Import der Module
#
import pandas
import numpy as np
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


#
# Einstellen der Grösse von Diagrammen
#
plt.rcParams['figure.figsize'] = [16, 9]

In [ ]:
# 
# Laden der Daten
# 
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
df = pandas.read_csv('data/iris/iris.csv', names=names)

In [ ]:
#
# Anzeige von Samples
#
print(df.head())

In [ ]:
#
# Prüfen der Verteilung der Klassen
#
print(df.groupby('class').size())

In [ ]:
# 
# Aufteilen in Training Daten und Testdaten
#
array = df.values
X = array[:,0:4]
Y = array[:,4]
validation_size = 0.40
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=43)

In [ ]:
#
# Trainieren eines einfachen classifiers der kNN Methode
#

knn_classifier = KNeighborsClassifier(n_neighbors=5,metric='euclidean')
knn_classifier.fit(X_train, Y_train)

In [ ]:
#
# Schätzung und Prüfung der Qualität
#
predictions = knn_classifier.predict(X_validation)
print('Accuracy: {}'.format(accuracy_score(Y_validation, predictions)))

In [ ]:
#
# Einfache confusion Matrix anzeigen
#
print(confusion_matrix(Y_validation, predictions))

# Test einer anderen Methode

In [ ]:
#
# Test eines decission tree classifiers
#
tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(X_train, Y_train)

In [ ]:
#
# Schätzung und Prüfung der Qualität
#
predictions = tree_classifier.predict(X_validation)
print('Accuracy: {}'.format(accuracy_score(Y_validation, predictions)))

In [ ]:
#
# Einfache confusion Matrix anzeigen
#
print(confusion_matrix(Y_validation, predictions))

# Test einer Reihe von Methoden gleichzeitig

In [ ]:
#
# Test einer Reihe von Modellen gleichzeitig
#
scoring = 'accuracy'

# Modelle anlegen
models = []
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
models.append(('RFT', RandomForestClassifier(max_depth=3, random_state=0)))
models.append(('NN20', MLPClassifier(hidden_layer_sizes=(10,10),random_state=1,max_iter=600,activation='relu')))

# Testen mit mehreren unterschiedlichen Aufteilungen der Daten
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=42,shuffle=True)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    print("Modell {}: accuracy {:.3f} (deviation {:.3f})".format(name, cv_results.mean(), cv_results.std()))

# Analyse der Daten

In [ ]:
#
# Analyse der Verteilung der Werte in den Features
#
df.hist()
plt.show()

In [ ]:
#
# Analyse der Verteilung der Features in Bezug auf die Klassenzuordnung
#
sns.pairplot(df, hue='class')